<a href="https://colab.research.google.com/github/nairsatish/4540/blob/master/demistifying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing required libraries
import numpy as np
import pandas as pd
import tqdm as tqdm
from keras.datasets import mnist

#loading dataset
(x_train, y_train), (x_test,y_test) = mnist.load_data()

#selecting a subset of data (200 images)
x_train = x_train[:200]
y=y_train[:200]

X = x_train.T
X = X/255

y.resize((200,1))
y = y.T
#checking value
pd.Series(y[0]).value_counts()
